# Вариант 68

## Задание 1

In [1]:
import numpy as np
np.set_printoptions(formatter={'float_kind': "{:.3f}".format})

In [2]:
def diagonal_to_matrix(vec: np.ndarray):
    size = len(vec)
    mtx = np.zeros((size, size))
    np.fill_diagonal(mtx, vec)
    return mtx


# 1
def transition_probability(mtx: np.ndarray, k: int):
    """Вероятность перехода из x в y за k шагов"""
    return mtx ** k

In [3]:
# 4
def first_return_probability(mtx: np.ndarray, k: int):
    """Вероятность первого возвращения на k-ом шаге"""
    f_k_jj = first_return_probability

    def p_k_jj(_mtx: np.ndarray, _k: int):
        return np.diagonal(transition_probability(_mtx, _k))

    if k == 1:
        return p_k_jj(mtx, 1)
    elif k <= 0:
        return np.zeros(3)

    m = k - 1
    return p_k_jj(mtx, k) - sum(f_k_jj(mtx, m - i) * p_k_jj(mtx, k - (m - i)) for i in range(m))


# 4.1
def avg_time_return(mtx: np.ndarray):
    """Среднее время возвращения"""
    return sum(t * first_return_probability(mtx, t) for t in range(1, 20))  # очень долго считает: на каждый элемент по 2 рекурсии


# 4.2
def last_return_probability(mtx: np.ndarray, k: int):
    """Вероятность возвращения не позднее чем за k шагов"""
    return sum(first_return_probability(mtx, t) for t in range(1, k + 1))

In [4]:
test_mtx = np.matrix([
    [0.4, 0.3, 0.3],
    [0.2, 0.5, 0.3],
    [0.1, 0.2, 0.7]
])
k = 4

In [5]:
# 4
first_return = diagonal_to_matrix(first_return_probability(test_mtx, k))
first_return

array([[0.060, 0.000, 0.000],
       [0.000, 0.066, 0.000],
       [0.000, 0.000, 0.044]])

In [6]:
# 4.1
avg_time_to_return = avg_time_return(test_mtx)
avg_time_to_return

array([4.212, 3.074, 1.989])

In [7]:
# 4.2
last_return = diagonal_to_matrix(last_return_probability(test_mtx, k))
last_return

array([[0.622, 0.000, 0.000],
       [0.000, 0.773, 0.000],
       [0.000, 0.000, 0.897]])

### Задание 2

In [8]:
def create_matrix(n, m, la, mu):
    size = n + m + 1
    matrix = np.zeros((size, size))
    np.fill_diagonal(matrix[:, 1:], la)
    np.fill_diagonal(matrix[1:, :], [*[i * mu for i in range(1, m)], *[m * mu for _ in range(n + 1)]])
    return matrix


def steady_state_probability(mtx):
    size = len(mtx)
    matrix_ = np.copy(matrix)
    matrix_ = matrix_.transpose()
    np.fill_diagonal(matrix_, [-sum(matrix_[:, i]) for i in range(size)])
    matrix_[-1, :] = 1

    vec_b = np.zeros(size)
    vec_b[-1] = 1
    return np.linalg.inv(matrix_).dot(vec_b)

In [9]:
la = 45
mu = 11
m = 5
n = 13

In [10]:
matrix = create_matrix(n, m, la, mu)

In [11]:
vec_p = steady_state_probability(matrix)
vec_p

array([0.012, 0.048, 0.097, 0.133, 0.136, 0.111, 0.091, 0.074, 0.061,
       0.050, 0.041, 0.033, 0.027, 0.022, 0.018, 0.015, 0.012, 0.010,
       0.008])

In [12]:
p_n_m = vec_p[-1]

In [13]:
# вероятность отказа
p_dis = p_n_m
p_dis

0.008185054380859143

In [14]:
# относительная пропускная способность
q = 1 - p_n_m
q

0.9918149456191409

In [15]:
# абсолютная пропускная способность
A = (1 - p_n_m) * la
A

44.63167255286134

In [16]:
# средняя длина очереди
len_queue = sum((i * vec_p[m + i]) for i in range(1, n + 1))
len_queue

2.0699645979756607

In [17]:
# среднее время в очереди
t_queue = sum(((i + 1) / (m * mu) * vec_p[m + i]) for i in range(n))
t_queue

0.04599921328834801

In [18]:
# среднее кол-во занятых каналов
channel_number = sum((i * vec_p[i]) for i in range(1, m + 1)) + sum((m * vec_p[i]) for i in range(m + 1, m + n + 1))
channel_number

4.057424777532847

In [19]:
# сумма уст вероятностей от 0 до m-1
sum_prob_state = sum(vec_p[:m])
sum_prob_state

0.42541710578449154

In [20]:
# среднее время простоя СМО
ne_t = 1 / np.sum(matrix, axis=1)
ne_t

array([0.022, 0.018, 0.015, 0.013, 0.011, 0.010, 0.010, 0.010, 0.010,
       0.010, 0.010, 0.010, 0.010, 0.010, 0.010, 0.010, 0.010, 0.010,
       0.018])